In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [46]:
import numpy as np
import pandas as pd
import yfinance as yf

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

import wavy
from wavy.utils import reverse_pct_change

In [3]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol

In [4]:
msft = yf.Ticker("MSFT")

In [5]:
# get historical market data from 2005 on
hist = msft.history(period="max", start="2005-01-01")

In [6]:
hist = hist[['Open', 'High', 'Low', 'Close']]
df = hist.pct_change()

In [7]:
print(len(df) - len(df.dropna()))
df.dropna(inplace=True)

1


In [92]:
x, y = wavy.create_panels(df, lookback=1, horizon=1)

# predict if the highest price > 0
y = (y['High'] > 0).astype(int)

In [98]:
base = wavy.models.BaselineShift(x=x, y=y, model_type='classification')
base.fit()
base.score()

../wavy/models.py:180: UserWarning: Filling nan values with 0
  warnings.warn(f"Filling nan values with {self.fillna}")


99/99 [==============================] - 0s 1ms/step - loss: 7.0020 - auc: 0.5434 - accuracy: 0.5435 - val_loss: 7.4243 - val_auc: 0.5107 - val_accuracy: 0.5159


,train,val,test
loss,7.002033,7.424325,6.824303
auc,0.543387,0.510714,0.551269
accuracy,0.543458,0.515924,0.555046


In [111]:
lr = wavy.models.LogisticRegression(x=x['Close'], y=y)
lr.fit(epochs=100, verbose=0)
lr.score()

,train,val,test
loss,0.680026,0.683247,0.682601
auc,0.676075,0.661115,0.651367
accuracy,0.611270,0.596178,0.573394


In [130]:
x_data = x['High'].as_dataframe().reset_index(drop=True)
y_data = y['High'].as_dataframe().reset_index(drop=True)

data = pd.concat([x_data, y_data], axis=1)

In [131]:
data.corr()

,High,High
High,1.000000,0.065631
High,0.065631,1.000000


In [94]:
dense = wavy.models.DenseModel(x=x, y=y, model_type='classification')
dense.fit(epochs=50)
dense.score()

Epoch 1/50
99/99 [==============================] - 0s 2ms/step - loss: 0.6912 - auc: 0.6052 - accuracy: 0.5864 - val_loss: 0.6904 - val_auc: 0.6099 - val_accuracy: 0.6038
Epoch 2/50
99/99 [==============================] - 0s 538us/step - loss: 0.6894 - auc: 0.6176 - accuracy: 0.5778 - val_loss: 0.6887 - val_auc: 0.6192 - val_accuracy: 0.6102
Epoch 3/50
99/99 [==============================] - 0s 542us/step - loss: 0.6874 - auc: 0.6312 - accuracy: 0.5890 - val_loss: 0.6871 - val_auc: 0.6284 - val_accuracy: 0.5962
Epoch 4/50
99/99 [==============================] - 0s 521us/step - loss: 0.6849 - auc: 0.6264 - accuracy: 0.5848 - val_loss: 0.6837 - val_auc: 0.6270 - val_accuracy: 0.6255
Epoch 5/50
99/99 [==============================] - 0s 511us/step - loss: 0.6822 - auc: 0.6253 - accuracy: 0.5855 - val_loss: 0.6803 - val_auc: 0.6324 - val_accuracy: 0.6280
Epoch 6/50
99/99 [==============================] - 0s 511us/step - loss: 0.6784 - auc: 0.6462 - accuracy: 0.6084 - val_loss: 0.6808

,train,val,test
loss,0.631498,0.647224,0.660955
auc,0.696106,0.694878,0.662656
accuracy,0.644062,0.662420,0.610092


In [27]:
x[0]

,Open,High,Low,Close
Date,,,,
2005-01-04,0.002612,5.565464e-03,0.000375,0.003739
2005-01-05,-0.001116,6.012214e-08,0.003751,-0.002235
2005-01-06,0.000373,-1.475732e-03,-0.004484,-0.001120
2005-01-07,-0.001117,-6.282067e-03,-0.000750,-0.002990
2005-01-10,-0.008203,-1.115739e-03,-0.003005,0.004874
2005-01-11,0.003383,-1.489672e-03,0.002637,-0.002612
2005-01-12,0.002997,1.118568e-03,0.000376,0.001871
2005-01-13,-0.003362,-1.861895e-03,-0.017280,-0.019044
2005-01-14,-0.010495,-1.305981e-02,-0.004587,-0.005710


In [39]:
x_max = x.max().to_frame().transpose()

x_max.rename(columns={'Open': 'max_open', 'High': 'max_high', 'Low': 'max_low', 'Close': 'max_close'}, inplace=True)

x_min = x.min().to_frame().transpose()

x_min.rename(columns={'Open': 'min_open', 'High': 'min_high', 'Low': 'min_low', 'Close': 'min_close'}, inplace=True)


x_mean = x.mean().to_frame().transpose()

x_mean.rename(columns={'Open': 'mean_open', 'High': 'mean_high', 'Low': 'mean_low', 'Close': 'mean_close'}, inplace=True)


x_std = x.std().to_frame().transpose()

x_std.rename(columns={'Open': 'std_open', 'High': 'std_high', 'Low': 'std_low', 'Close': 'std_close'}, inplace=True)

# x_last = x.iloc[-1].to_frame().transpose()

# x_last.rename(columns={'Open': 'last_open', 'High': 'last_high', 'Low': 'last_low', 'Close': 'last_close'}, inplace=True)

In [40]:
from wavy import Panel

xx = Panel([pd.concat(ls, axis=1) for ls in zip(x_max, x_min, x_mean, x_std)])

In [41]:
xx[0]

,max_open,max_high,max_low,max_close,min_open,min_high,min_low,min_close,mean_open,mean_high,mean_low,mean_close,std_open,std_high,std_low,std_close
0,0.003383,0.005565,0.003751,0.007657,-0.014015,-0.01306,-0.01728,-0.019044,-0.002894,-0.002238,-0.003065,-0.001557,0.006067,0.004894,0.006107,0.007401


In [75]:
lr.score()

,train,val,test
loss,0.689902,0.691065,0.691444
auc,0.559031,0.585592,0.584791
accuracy,0.539075,0.559387,0.524138


In [ ]:
# SHALLOW

In [63]:
x_train = xx.train.as_dataframe().reset_index(drop=True)
y_train = y.train.as_dataframe().reset_index(drop=True).values[:, 0]

x_val = xx.val.as_dataframe().reset_index(drop=True)
y_val = y.val.as_dataframe().reset_index(drop=True).values[:, 0]

x_test = xx.test.as_dataframe().reset_index(drop=True)
y_test = y.test.as_dataframe().reset_index(drop=True).values[:, 0]

In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

lr = LogisticRegression()
lr.fit(x_train, y_train)

rf = RandomForestClassifier()
rf.fit(x_train, y_train)

RandomForestClassifier()

In [82]:
roc_auc_score(y_val, lr.predict_proba(x_val)[:, 1])

0.546182777961069

In [81]:
roc_auc_score(y_val, rf.predict_proba(x_val)[:, 1])

0.6046123391619928